In [1]:
import pandas as pd
import matplotlib.pyplot as plt

**Units**
* values in CO2_totals_by_country sheet are expressed in Mt CO2/yr 
* values in CO2_by_sector_and_country sheet are expressed in Mt CO2/yr
* values in fossil_CO2_per_GDP_by_country sheet are expressed in t CO2/kUSD/yr
* values in fossil_CO2_per_capita_by_countr sheet are expressed in t CO2/cap/yr

In [2]:
desired_regions = [
    'AT',
    'BE',
    'BG',
    'CH',
    'CZ',
    'DK',
    'DE',
    'EE',
    'ES',
    'FR',
    'FI',
    'GR',
    'HR',
    'HU',
    'IE',
    'IT',
    'LT',
    'LU',
    'LV',
    'NL',
    'NO',
    'PL',
    'PT',
    'RO',
    'SI',
    'SE',
    'SK',
    'UK',
]

In [3]:
regions_to_EDGAR = {
    'AT' : 'AUT',
    'BE' : 'BEL',
    'BG' : 'BGR',
    'CH' : 'CHE',
    'CZ' : 'CZE',
    'DK' : 'DNK',
    'DE' : 'DEU',
    'EE' : 'EST',
    'ES' : 'ESP',
    'FR' : 'FRA',
    'FI' : 'FIN',
    'GR' : 'GRC',
    'HR' : 'HRV',
    'HU' : 'HUN',
    'IE' : 'IRL',
    'IT' : 'ITA',
    'LT' : 'LTU',
    'LU' : 'LUX',
    'LV' : 'LVA',
    'NL' : 'NLD',
    'NO' : 'NOR',
    'PL' : 'POL',
    'PT' : 'PRT',
    'RO' : 'ROU',
    'SI' : 'SVN',
    'SE' : 'SWE',
    'SK' : 'SVK',
    'UK' : 'GBR'
}

In [4]:
EDGAR_to_regions = {v: k for k, v in regions_to_EDGAR.items()}

In [6]:
emissions = pd.read_csv(
    'data/annual-co2-emissions-per-country.csv',
    sep=','
).set_index(
    ['Entity','Code','Year']
).rename(
    EDGAR_to_regions
).loc[
    (slice(None),desired_regions,slice(None))
]

In [7]:
years = list(range(1901,2022))

In [8]:
len(years)

121

In [9]:
missing_data = []
for country in desired_regions:
    data_points = emissions.reset_index().query("Year > 1900").query("Code == @country").Year.count()
    if data_points != len(years):
        print(country + ': ' 'Only ' + str(data_points) + ' years of data')
        missing_data.append(country)

GR: Only 120 years of data
HR: Only 114 years of data
IE: Only 108 years of data
LU: Only 77 years of data
SI: Only 114 years of data


In [10]:
missing_data

['GR', 'HR', 'IE', 'LU', 'SI']

In [11]:
emissions.query("Code == 'GR' & Year > 1900").reset_index().Year.unique()

array([1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1910, 1911,
       1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923,
       1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934,
       1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945,
       1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956,
       1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967,
       1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978,
       1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989,
       1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
       2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021])

In [12]:
emission_1900 = emissions.query("Year > 1900")

In [13]:
(
    emission_1900
    .unstack()
    .T
    .droplevel(axis=1, level=0)
    .interpolate()
    .fillna(method='bfill')
    .T
    .unstack()
    .to_frame()
    .reset_index()
    .groupby('Code').sum()
    .drop(columns={'Year'})
    .reset_index()
    .sort_values(by = 0)
    .to_csv('data/emissions_1900_2021.tsv',sep='\t')
)

/tmp/ipykernel_603701/3050030641.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .groupby('Code').sum()
